# Work space vorbereiten

Importieren unsere Bibliotheken wie NLTK, Codecs und HanoverTagger 

In [16]:
import codecs 
import nltk
from HanTa import HanoverTagger as ht
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

In [2]:
words = []

with codecs.open('woerter/NN_masc.txt','r','utf8') as datei:
    for word in datei:
        words.append((word.strip(),'m'))# m für Maskulin der
with codecs.open('woerter/NN_fem.txt','r','utf8') as datei:
    for word in datei:
        words.append((word.strip(),'f'))# f für Femenin die 
with codecs.open('woerter/NN_neutr.txt','r','utf8') as datei:
    for word in datei:
        words.append((word.strip(),'n'))# n für Neutral


In [19]:
words[1:3]

[(['##n', '#nu', 'nut', 'utz', 'tze', 'zer', 'er#', 'r##'], 'm'),
 (['##v',
   '#ve',
   'ver',
   'ert',
   'rte',
   'tei',
   'eid',
   'idi',
   'dig',
   'igu',
   'gun',
   'ung',
   'ngs',
   'gsb',
   'sbu',
   'bud',
   'udg',
   'dge',
   'get',
   'et#',
   't##'],
  'n')]

In [3]:
len(words)

8616

# Punkt 3    Trigramme extrahieren

bevor ich meine Datein für Test und Training Daten trenne, wollte ich zuerst den Punkt 3(Trigramme extrahieren) erledigen, damit ich vom Anfang die Wörter ausrichten kann.  

In [4]:
def ngram(string,n):
    liste = []
    if n < len(string):
        for p in range(len(string) - n + 1) :
            tg = string[p:p+n]
            liste.append(tg)
    return liste
def xgram(string):
    return [w for w in ngram(string.lower(),3)]

# Punkt 2    Wörter erweitern

wir mussen den Wörtern Sonderzeichnen einfügen, damit wir gute Ergebnisse bei den Trigramme bekommen. In folgendem Code extrahieren wir die trigramme und erweitern wir die Wörte.

In [5]:
words=[(xgram('##'+a+'##'),b) for (a,b) in words]

In [6]:
words[20]

(['##k', '#ko', 'kon', 'onz', 'nze', 'zer', 'ern', 'rn#', 'n##'], 'm')

# Punkt 1 : Test und Training Daten trennen 

Unser Daten sin ca. 8600 Datei. wir nehmen für die Training 80% davon. Mit den Anderen testen wir unser Model.  

In [7]:
import random

random.shuffle(words)
train_data = words[:6880]
test_data = words[6880:]

# Anteil der Wörter

Durch die  Funktion (features_from_text) können wir wisse,was der Anteil eines Wortes/ eines Trigramms ist. Das können entdecken, wenn wir den Anzahl des Wortes durch den Anzahl aller Textwörter teilen.

In [10]:
from collections import Counter

def features_from_text(text):
    wordcounts = Counter()
    tlen = 0
    wordcounts.update(text)
    tlen +=len(text)
    return {w:wordcounts[w]/tlen for w in wordcounts}

In [11]:
#poslist = ["NN","NE"]
test_data= [(features_from_text(text), cl)for text,cl in test_data]
train_data= [(features_from_text(tex,poslist),cl) for text,cl in train_data]


# Häufigkeit aller vorkommenden Trigramme

Damit wir die Trigramme, die häufig in der Datein vorkommen, entdecken zu können, bauen wir ein Counter und zahlen wir, wie oft ein Wort bzw. ein Trigramm kommt.

In [12]:
docfreq = Counter()
for (wfreq,c) in train_data:
    docfreq.update(wfreq.keys())
docfreq.most_common(20)

[('t##', 1162),
 ('g##', 1065),
 ('ung', 1058),
 ('e##', 1056),
 ('r##', 941),
 ('ng#', 903),
 ('er#', 746),
 ('##s', 728),
 ('sch', 712),
 ('n##', 687),
 ('##a', 555),
 ('##b', 520),
 ('##k', 448),
 ('ter', 408),
 ('ver', 407),
 ('##g', 398),
 ('eit', 371),
 ('##f', 368),
 ('##v', 367),
 ('##m', 357)]

# Einbauen des Trainingsmodel 

# Punkt 4 :

1 - Importieren die Bibioltek


2 - Die häufigste vorkommende Trigramme rausnehmen, damit wir den Netz trainieren.

In [13]:
from sklearn import linear_model, datasets


allfeatures = [w for w in docfreq if docfreq[w] > 4]

def make_feat_vec(featmap,featlist):
    vec = []
    for f in featlist:
        vec.append(featmap.get(f,0.0))
    return vec

train_vec =  [make_feat_vec(feats,allfeatures) for feats,cls in train_data]
train_label = [cls for feats,cls in train_data]

3 - fitten wir das Modell durch die Trainingsdaten

In [14]:
logreg = linear_model.LogisticRegression(C=1e9, verbose = True, max_iter=4000 )
logreg.fit(train_vec,train_label)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.7min finished


LogisticRegression(C=1000000000.0, max_iter=4000, verbose=True)

4- Testen wir das Model mit den Testdaten

In [15]:
test_vec = [make_feat_vec(feats,allfeatures) for feats,cls in test_data]
test_label = [cls for feats,cls in test_data]

pred_label = list(logreg.predict(test_vec))

# Punkt 5 :Bewertung des Model

Wir evaluieren das Model und wiessen, wie es correct ist?

In [17]:
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

76.8 Prozent korrekt


Ried